### Task 4.1 - Visualization - Interactive Map

In [1]:
import pandas as pd
import folium
from folium.features import DivIcon
from folium.plugins import MarkerCluster

In [2]:
df = pd.read_csv("datasets/falcon9_dataset.csv")
df.head()

,FlightNumber,Date,Hour,Minute,BoosterVersion,PayloadMass,Orbit,LaunchSite,Serial,Block,Longitude,Latitude,Outcome,LandingPlace,Year,Month,Day,Flights,Reused
0,1,2010-06-04,18,45,Falcon 9,0.0,LEO,CCSFS SLC 40,B0003,1.0,-80.577366,28.561857,0,Ocean,2010,6,4,1,False
1,2,2010-12-08,15,43,Falcon 9,2115.0,ISS,CCSFS SLC 40,B0004,1.0,-80.577366,28.561857,0,Ocean,2010,12,8,1,False
2,6,2013-09-29,16,0,Falcon 9,500.0,PO,VSFB SLC 4E,B1003,1.0,-120.610829,34.632093,0,Ocean,2013,9,29,1,False
3,9,2014-04-18,19,25,Falcon 9,2296.0,ISS,CCSFS SLC 40,B1015,1.0,-80.577366,28.561857,1,Ocean,2014,4,18,1,False
4,10,2014-07-14,15,15,Falcon 9,1316.0,LEO,CCSFS SLC 40,B1018,1.0,-80.577366,28.561857,1,Ocean,2014,7,14,1,False


In [3]:
success_rate = round((df.groupby(["LaunchSite", "Latitude", "Longitude"])["Outcome"].sum()/df.groupby(["LaunchSite"])["Outcome"].count())*100,2)
success_rate = pd.DataFrame(success_rate).reset_index()
success_rate.columns = ["LaunchSite", "Latitude", "Longitude", "SuccessRate"]
success_rate

,LaunchSite,Latitude,Longitude,SuccessRate
0,CCSFS SLC 40,28.561857,-80.577366,91.27
1,KSC LC 39A,28.608058,-80.603956,98.28
2,VSFB SLC 4E,34.632093,-120.610829,95.83


In [4]:
# each site location and its success rate information
site_map = folium.Map(location=[39.579262, -101.373816], zoom_start=4)
colors = ['#0000AA', '#FF6912', '#FF3333']
text_positions = [(0,0), (0,60), (0,60)]

for site, lat, lon, rate, colour, text_pos in zip(success_rate.LaunchSite, success_rate.Latitude, success_rate.Longitude, success_rate.SuccessRate, colors, text_positions):
    tooltip=f"<h5><b>Launch site: {site}</b>" + f"<h6>Landing success rate: {rate}%"
    site_map.add_child(folium.Circle(location=[lat, lon], color=colour, radius=1100, fill=True, tooltip=tooltip))
    folium.Marker(location=[lat,lon], icon=DivIcon(icon_size=(20,20), icon_anchor=text_pos, html=f'<div style="font-size: 10pt; color:{colour}">{site}</div>')).add_to(site_map)
    
site_map

In [5]:
# mark success/fail langings for each site
marker_cluster = MarkerCluster()

for lat, lon, number, date, hour, minute, mass, orbit, outcome in zip(df.Latitude, df.Longitude, df.FlightNumber, df.Date, df.Hour, df.Minute, df.PayloadMass, df.Orbit, df.Outcome):   
    if minute < 10:
        popup = f"<h5><b>Flight number: {number}</b>" + f"<br><h5>Launch time: {date} {hour}:0{minute}" + f"<br><h5>Payload Mass: {mass}kg" + f"<br><h5>Orbit: {orbit}"
    else:
        popup = f"<h5><b>Flight number: {number}</b>" + f"<br><h5>Launch time: {date} {hour}:{minute}" + f"<br><h5>Payload Mass: {mass}kg" + f"<br><h5>Orbit: {orbit}"
    
    if outcome:
        marker_cluster.add_child(folium.Marker(location=[lat, lon],
                                               popup=folium.Popup(popup + f"<br><h5>Landing outcome: Succeed",
                                                                  min_width=200, max_width=200),
                                                                  icon=folium.Icon(color="lightblue", icon_color='white',icon='fa-regular fa-circle-check', prefix='fa')))
    else:
        marker_cluster.add_child(folium.Marker(location=[lat, lon],
                                               popup=folium.Popup(popup + f"<br><h5>Landing outcome: Failed",
                                                                  min_width=200, max_width=200),
                                                                  icon=folium.Icon(color="lightred", icon_color='white',icon='fa-regular fa-circle-xmark', prefix='fa')))

marker_cluster.add_to(site_map)

site_map